In [207]:
import pandas as pd
import openpyxl as opx
import xlrd
import os

class sht_merge:
    def __init__(self):
        pass
    
    def rd(self):
        sht_0=pd.read_excel("ttttt.xlsm",sheet_name=0)
        sht_1=pd.read_excel("ttttt.xlsm",sheet_name=1,usecols=[0,1,2,3,4,5,6,7,8])
        sht_2=self.slct_valid() #先整理出有效的律所列表
        sht_3=pd.read_excel("ttttt.xlsm",sheet_name=3)
       

        res_1=sht_0.sort_values(by="案件名称")
        cls_2=sht_1.loc[sht_1['审级']==2]  #二审
        cls_3=sht_1.loc[sht_1['审级']==3]  #再审
        cls_4=sht_1.loc[sht_1['审级']==4]  #执行

        
        res=pd.merge(res_1,cls_2,left_on='案号',right_on='一审案号',how="left",suffixes=["","_二审"]) \
                    .drop(columns=["一审案号","案件名称_二审","审级","主办_二审","副办_二审"])
        res=pd.merge(res,cls_3,left_on='案号',right_on='一审案号',how="left",suffixes=["","_再审"]) \
                    .drop(columns=["一审案号","案件名称_再审","审级","审理情况_再审","主办法官_再审","管辖法院_再审","主办_再审","副办_再审"])  
        res=pd.merge(res,sht_2,left_on='案件名称',right_on='一审案件名称',how="left") \
                    .drop(columns=["一审案件名称"])

        
        order=["户数","案件数","案件名称","起诉金额（元）","财务代偿余额（元）","抵押物情况","主办","副办","诉讼状态" \
                ,"代理方式","律所名称","辅助机构" \
                ,"案号","案号_二审","案号_再审","执行案号","起诉日期","保全申请" \
                ,"立案日期","保全进度","一审情况","执行情况","一审管辖法院","一审主办法官" \
                ,"审理情况","管辖法院","主办法官" \
                ,"执行情况","执行法院","执行主办法官" \
                ,"备注"]
        
        res=res[order]
        res=res.rename(columns={"案号":"一审案号","案号_二审":"二审案号","案号_再审":"再审案号" \
                           ,"审理情况":"二审审理情况","管辖情况":"二审管辖情况","主办法官":"二审主办法官" \
                           })

        print(res)
# #         print(res_1,'\n',res_2to1,'\n',res_3to1)
#         k=res.drop(columns=["序号_x","序号_y","公司_y","一审案号_x","序号_x","一审案号_y","序号_y","公司_y"])
#         print(k)
        self.res=res
    
    #处理代理机构表，挑选出有效的律所
    def slct_valid(self):

        sht_2=pd.read_excel("ttttt.xlsm",sheet_name=2)

        dl_1=sht_2.loc[sht_2["一审代理律所名称"]==sht_2["目前有效"]] \
            .drop(columns=["二审代理律所名称","再审代理律所名称","执行代理律所名称"]) \
            .rename(columns={"一审代理律所名称":"律所名称"})

        dl_2=sht_2.loc[sht_2["二审代理律所名称"]==sht_2["目前有效"]] \
            .drop(columns=["一审代理律所名称","再审代理律所名称","执行代理律所名称"]) \
            .rename(columns={"二审代理律所名称":"律所名称"})

        dl_3=sht_2.loc[sht_2["再审代理律所名称"]==sht_2["目前有效"]] \
            .drop(columns=["一审代理律所名称","二审代理律所名称","执行代理律所名称"]) \
            .rename(columns={"再审代理律所名称":"律所名称"})

        dl_4=sht_2.loc[sht_2["执行代理律所名称"]==sht_2["目前有效"]] \
            .drop(columns=["一审代理律所名称","二审代理律所名称","再审代理律所名称"]) \
            .rename(columns={"执行代理律所名称":"律所名称"})

        dl=pd.concat([dl_1,dl_2,dl_3,dl_4]).drop(columns=["目前有效"])
        return dl    
    def wt(self):
        wt=pd.ExcelWriter('output.xlsx')
        self.res.to_excel(wt,index=False)
        wt.save()
    
class wt_excel:
    def __init__(self):
        pass
    
    def wt(self):
        wb = opx.load_workbook("output.xlsx")
        sht = wb['Sheet1']
        print(sht.max_row)
        e="A"+str(sht.max_row)
        
        n=1
        for rows in sht["A1":e]:
            for cells in rows:
                print(cells.value,n)
                n+=1
                
                

#         sht.merge_cells("G2:G5")
#         wb.save("merged.xlsx")
        
        
        
if __name__=="__main__":
    luo=sht_merge()
    luo.rd()
    luo.wt()
    
#     wt=wt_excel()
#     wt.wt()




   户数  案件数 案件名称  起诉金额（元）  财务代偿余额（元）  抵押物情况  主办  副办  诉讼状态  代理方式  ...  执行情况  \
0 NaN  NaN    a      100    1000000    NaN NaN NaN   NaN   NaN  ...    良好   
1 NaN  NaN    a      200    1000000    NaN NaN NaN   NaN   NaN  ...    一般   
2 NaN  NaN    a      300    1000000    NaN NaN NaN   NaN   NaN  ...     中   
3 NaN  NaN    b      100    2000000    NaN NaN NaN   NaN   NaN  ...     差   
4 NaN  NaN    b       50    2000000    NaN NaN NaN   NaN   NaN  ...     中   
5 NaN  NaN    c       30    3000000    NaN NaN NaN   NaN   NaN  ...     差   
6 NaN  NaN    c       67    3000000    NaN NaN NaN   NaN   NaN  ...     差   

   一审管辖法院 一审主办法官 二审审理情况 管辖法院  二审主办法官  执行情况  执行法院  执行主办法官  备注  
0     NaN    NaN    NaN  NaN       我    良好   NaN     NaN NaN  
1     NaN    NaN    NaN  NaN     NaN    一般   NaN     NaN NaN  
2     NaN    NaN    NaN  NaN     NaN     中   NaN     NaN NaN  
3     NaN    NaN    NaN  NaN     NaN     差   NaN     NaN NaN  
4     NaN    NaN    NaN  NaN       我     中   NaN     NaN NaN  
5    

In [190]:
import pandas as pd
import openpyxl as opx
import xlrd
import os

def slct_lv():

    sht_2=pd.read_excel("ttttt.xlsm",sheet_name=2,usecols=[0,1,2,3,4,5,6,7,8])

    dl_1=sht_2.loc[sht_2["一审代理律所名称"]==sht_2["目前有效"]] \
        .drop(columns=["二审代理律所名称","再审代理律所名称","执行代理律所名称"]) \
        .rename(columns={"一审代理律所名称":"律所名称"})

    dl_2=sht_2.loc[sht_2["二审代理律所名称"]==sht_2["目前有效"]] \
        .drop(columns=["一审代理律所名称","再审代理律所名称","执行代理律所名称"]) \
        .rename(columns={"二审代理律所名称":"律所名称"})

    dl_3=sht_2.loc[sht_2["再审代理律所名称"]==sht_2["目前有效"]] \
        .drop(columns=["一审代理律所名称","二审代理律所名称","执行代理律所名称"]) \
        .rename(columns={"再审代理律所名称":"律所名称"})

    dl_4=sht_2.loc[sht_2["执行代理律所名称"]==sht_2["目前有效"]] \
        .drop(columns=["一审代理律所名称","二审代理律所名称","再审代理律所名称"]) \
        .rename(columns={"执行代理律所名称":"律所名称"})

    dl=pd.concat([dl_1,dl_2,dl_3,dl_4]).drop(columns=["目前有效"])
    return dl
# print(sht_2)

print(slct_lv())

  一审案件名称  代理方式 律所名称  辅助机构  财务代偿余额（元）
0      a   NaN    一   NaN    1000000
2      c   NaN    二   NaN    3000000
1      b   NaN    三   NaN    2000000
